## Importing Libraries


In [ ]:
!pip install -q mediapipe==0.10.7

In [2]:
import os
import gc
import cv2
import json
import time
import shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from IPython.display import clear_output, FileLink

## Data Preparation

In [2]:
train_path = 'asl-citizen-1707816139686\ASL_Citizen\splits\\train.csv'
val_path = 'asl-citizen-1707816139686/ASL_Citizen/splits/val.csv'
test_path = 'asl-citizen-1707816139686\ASL_Citizen\splits\\test.csv'

train_data = pd.read_csv(train_path)
val_data = pd.read_csv(val_path)
test_data = pd.read_csv(test_path)

In [3]:
train_data['split'] = 'train'
val_data['split'] = 'val'
test_data['split'] = 'test'

train_data = train_data[['Video file', 'Gloss', 'split']]
val_data = val_data[['Video file', 'Gloss', 'split']]
test_data = test_data[['Video file', 'Gloss', 'split']]

data = pd.concat([train_data, val_data, test_data], ignore_index=True)
data = data.sample(frac=1, random_state=42, ignore_index=True)
data = data.rename(columns={'Gloss':'label', 'Video file': 'video'})
data.head()

,video,label,split
0,3241300306459223-SMELL MONEY.mp4,SMELLMONEY,test
1,21927612901885207-DRIVE.mp4,DRIVE,test
2,40551766570522196-PITCHER.mp4,PITCHER,train
3,07035019876456139-LICK ENVELOPE_2.mp4,LICKENVELOPE2,train
4,973653586698978-MAN.mp4,MAN,val


In [4]:
def extract_label(file_name):
    label_parts = file_name.split("-")[1].split(".")[0].replace("_", " ").split(" ")
    label_parts = [part for part in label_parts if part != '']
    if label_parts[0].startswith('seed'):
        label_parts[0] = label_parts[0][4:]
    if label_parts[-1].isdigit():
        label_parts = label_parts[:-1]
    label_parts = [part.lower() for part in label_parts]
    return ' '.join(label_parts)

In [5]:
data.insert(data.columns.get_loc('label') + 1, 'clean label', data['video'].apply(extract_label))
data.head()

,video,label,clean label,split
0,3241300306459223-SMELL MONEY.mp4,SMELLMONEY,smell money,test
1,21927612901885207-DRIVE.mp4,DRIVE,drive,test
2,40551766570522196-PITCHER.mp4,PITCHER,pitcher,train
3,07035019876456139-LICK ENVELOPE_2.mp4,LICKENVELOPE2,lick envelope,train
4,973653586698978-MAN.mp4,MAN,man,val


In [6]:
data[data['clean label'] == '']

,video,label,clean label,split
59741,4.7299129501965353e-7-seedSOUR.mp4,SOUR,,train


In [7]:
data.iloc[59741, data.columns.get_loc('clean label')] = 'sour'
data[data['clean label'] == '']

,video,label,clean label,split


In [8]:
data.to_csv("data.csv", index=False)

---

## Landmark detection and extraction

In [ ]:
import mediapipe as mp

### Including Face Landmarks

In [2]:
filtered_hand = list(range(21))

filtered_pose = [11, 12, 13, 14, 15, 16]

filtered_face = [0, 4, 7, 8, 10, 13, 14, 17, 21, 33, 37, 39, 40, 46, 52, 53, 54, 55, 58,
                 61, 63, 65, 66, 67, 70, 78, 80, 81, 82, 84, 87, 88, 91, 93, 95, 103, 105,
                 107, 109, 127, 132, 133, 136, 144, 145, 146, 148, 149, 150, 152, 153, 154,
                 155, 157, 158, 159, 160, 161, 162, 163, 172, 173, 176, 178, 181, 185, 191,
                 234, 246, 249, 251, 263, 267, 269, 270, 276, 282, 283, 284, 285, 288, 291,
                 293, 295, 296, 297, 300, 308, 310, 311, 312, 314, 317, 318, 321, 323, 324,
                 332, 334, 336, 338, 356, 361, 362, 365, 373, 374, 375, 377, 378, 379, 380,
                 381, 382, 384, 385, 386, 387, 388, 389, 390, 397, 398, 400, 402, 405, 409,
                 415, 454, 466, 468, 473]

num_hand = len(filtered_hand)
num_pose = len(filtered_pose)
num_face = len(filtered_face)

In [42]:
hands = mp.solutions.hands.Hands()
pose = mp.solutions.pose.Pose()
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

def get_frame_landmarks(frame):
    
    all_landmarks = np.zeros((num_hand * 2 + num_pose + num_face, 3))
    
    def get_hands(frame):
        results_hands = hands.process(frame)
        if results_hands.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
                if results_hands.multi_handedness[i].classification[0].index == 0: 
                    all_landmarks[:num_hand, :] = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]) # right
                else:
                    all_landmarks[num_hand:num_hand * 2, :] = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]) # left

    def get_pose(frame):
        results_pose = pose.process(frame)
        if results_pose.pose_landmarks:
            all_landmarks[num_hand * 2:num_hand * 2 + num_pose, :] = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_pose.pose_landmarks.landmark])[filtered_pose]
        
    def get_face(frame):
        results_face = face_mesh.process(frame)
        if results_face.multi_face_landmarks:
            all_landmarks[num_hand * 2 + num_pose:, :] = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_face.multi_face_landmarks[0].landmark])[filtered_face]
        
    with ThreadPoolExecutor(max_workers=3) as executor:
        executor.submit(get_hands, frame)
        executor.submit(get_pose, frame)
        executor.submit(get_face, frame)

    return all_landmarks

In [43]:
def get_video_landmarks(video_path):
    cap = cv2.VideoCapture(video_path)
    all_frame_landmarks = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_landmarks = get_frame_landmarks(frame)
        all_frame_landmarks.append(frame_landmarks)

    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.submit(cap.release)
        executor.submit(hands.reset)
        executor.submit(pose.reset)
        executor.submit(face_mesh.reset)
        
    return np.array(all_frame_landmarks)

In [44]:
def draw_landmarks(input_path, output_path, video_landmarks):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_landmarks = video_landmarks[frame_index]
        landmarks = [(int(x * width), int(y * height)) for x, y, _ in frame_landmarks]
        for x, y in landmarks:
            cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)
        out.write(frame)
        frame_index += 1

    cap.release()
    out.release()

### Excluding Face Landmarks (Better Results)

In [ ]:
filtered_hand = list(range(21))
filtered_pose = [0, 2, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

num_hand = len(filtered_hand)
num_pose = len(filtered_pose)

In [ ]:
hands = mp.solutions.hands.Hands()
pose = mp.solutions.pose.Pose()

In [ ]:
def get_frame_landmarks(frame):

    all_landmarks = np.zeros((num_hand * 2 + num_pose, 3))

    def get_hands(frame):
        results_hands = hands.process(frame)
        if results_hands.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
                if results_hands.multi_handedness[i].classification[0].index == 0:
                    all_landmarks[:num_hand, :] = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
                    )  # right
                else:
                    all_landmarks[num_hand : num_hand * 2, :] = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
                    )  # left

    def get_pose(frame):
        results_pose = pose.process(frame)
        if results_pose.pose_landmarks:
            all_landmarks[num_hand * 2 : num_hand * 2 + num_pose, :] = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_pose.pose_landmarks.landmark]
            )[filtered_pose]

    with ThreadPoolExecutor(max_workers=2) as executor:
        executor.submit(get_hands, frame)
        executor.submit(get_pose, frame)

    return all_landmarks

In [ ]:
def get_video_landmarks(video_path):
    cap = cv2.VideoCapture(video_path)
    all_frame_landmarks = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_landmarks = get_frame_landmarks(frame)
        all_frame_landmarks.append(frame_landmarks)

    with ThreadPoolExecutor(max_workers=3) as executor:
        executor.submit(cap.release)
        executor.submit(hands.reset)
        executor.submit(pose.reset)
        
    return np.array(all_frame_landmarks)

In [ ]:
def draw_landmarks(input_path, output_path, video_landmarks):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_landmarks = video_landmarks[frame_index]
        landmarks = [(int(x * width), int(y * height)) for x, y, _ in frame_landmarks]
        for x, y in landmarks:
            cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)
        out.write(frame)
        frame_index += 1

    cap.release()
    out.release()

## Data Encoding

In [51]:
txt_path = 'Test/completed.txt'
video_dir = 'asl-citizen-1707816139686\ASL_Citizen\\videos'
data_directory = 'Landmarks'
os.makedirs(data_directory, exist_ok=True)

In [46]:
labels = []
with open('Test/filtered.txt', 'r') as file:
    labels = file.read().splitlines()
    
len(labels)

530

In [47]:
data = pd.read_csv('data.csv')
data = data[data['clean label'].isin(labels)].reset_index(drop=True)
data.head()

,video,label,clean label,split
0,21927612901885207-DRIVE.mp4,DRIVE,drive,test
1,9666290416558401-NOON 2.mp4,NOON2,noon,val
2,33468636332219814-FOLLOW 1.mp4,FOLLOW2,follow,train
3,6768708457814387-SUNSET.mp4,SUNSET,sunset,train
4,8855248507178961-ENERGY.mp4,ENERGY,energy,train


In [48]:
len(data)

18030

In [53]:
try:
    completed_files = set()
    if os.path.exists(txt_path):
        with open(txt_path, 'r') as file:
            completed_files = {line.strip() for line in file}
            
    for i in tqdm(range(len(data)), ncols=100):
        video_name = data.iloc[i]['video'].split('.')[0]
        if video_name in completed_files: continue
        data_path = os.path.join(data_directory, f'{video_name}.npy')
        video_path = os.path.join(video_dir, f'{video_name}.mp4')

        try:
            video_landmarks = get_video_landmarks(video_path)
            np.save(data_path, video_landmarks)
            
            with open(txt_path, 'a') as file:
                file.write(f"{video_name}\n")
            completed_files.add(video_name)
            
        except Exception as e:
            print(f"\nError encoding {i}: {video_path}\n{e}")
            continue   
        clear_output(wait=True)

except KeyboardInterrupt:
    print("\nLoading process interrupted by user.")

100%|██████████████████████████████████████████████████████| 18030/18030 [11:42:10<00:00,  2.34s/it]


In [ ]:
landmarks_dict = {}

for filename in os.listdir(data_directory):
    if filename.endswith('.npy'):
        key = os.path.splitext(filename)[0]
        landmarks = np.load(os.path.join(data_directory, filename), allow_pickle=True)
        landmarks_dict[key] = landmarks

np.savez_compressed('landmarks.npz', **landmarks_dict)

## Label Filtering

    


In [ ]:
import fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')

In [ ]:
import fasttext
ft = fasttext.load_model('cc.en.300.bin')

In [28]:
data = pd.read_csv('data.csv')
labels = data['clean label'].unique().tolist()
len(data), len(labels)

(83399, 2299)

In [ ]:
labels_dir = 'Encoded Labels'
os.makedirs(labels_dir, exist_ok=True)

for i in tqdm(range(len(labels)), ncols=100):
    label_vector = ft.get_sentence_vector(labels[i])
    label_path = os.path.join(labels_dir, f'{labels[i]}.npy')
    np.save(label_path, label_vector)

In [ ]:
labels_dir = 'Encoded Labels'
encoded_labels = {}

for filename in os.listdir(labels_dir):
    if filename.endswith('.npy'):
        label = os.path.splitext(filename)[0]
        encoded_label = np.load(os.path.join(labels_dir, filename), allow_pickle=True)
        encoded_labels[label] = encoded_label
        
len(encoded_labels)

In [ ]:
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
word = 'taxi'
word_vector = ft.get_sentence_vector(word)

similarity_scores = []
for label, label_vector in encoded_labels.items():
    similarity_scores.append(cosine_similarity(word_vector, label_vector))

In [ ]:
bins = np.linspace(0, 1, 21)

plt.hist(similarity_scores, bins=bins, edgecolor='black')
plt.title(f'Cosine Similarity Distribution to "{word}"')
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')
plt.xticks(np.arange(0, 1.1, 0.1))
plt.grid(axis='y')
plt.show()

In [ ]:
threshold = 0.4
similar_words_set = set()

for label, label_vector in encoded_labels.items():
    similarity_score = cosine_similarity(word_vector, label_vector)
    if similarity_score > threshold:
        similar_words_set.add(label)
        
print(similar_words_set)

In [ ]:
taxi_words = [
    'hello',
    'goodbye',
    'help',
    'thank you',
    'sorry',
    'yes',
    'no',
    'stop',
    'go',
    'left',
    'right',
    'ahead',
    'around',
    'address',
    'destination',
    'time',
    'money',
    'cost',
    'lost',
    'map',
    'street',
    'road',
    'taxi',
    'car',
    'bus',
    'school',
    'university',
    'food',
    'fuel',
]

len(taxi_words)

In [ ]:
taxi_dir = 'Encoded Samples'
os.makedirs(taxi_dir, exist_ok=True)

for i in tqdm(range(len(taxi_words)), ncols=100):
    taxi_vector = ft.get_sentence_vector(taxi_words[i])
    taxi_path = os.path.join(taxi_dir, f'{taxi_words[i]}.npy')
    np.save(taxi_path, taxi_vector)

In [ ]:
taxi_dir = 'Encoded Samples'
taxi_vectors = []

for filename in os.listdir(taxi_dir):
    if filename.endswith('.npy'):
        taxi_vectors.append(np.load(os.path.join(taxi_dir, filename), allow_pickle=True))
        
len(taxi_vectors)

In [ ]:
threshold = 0.4
similar_words_set = set()

for label, label_vector in encoded_labels.items():
    for taxi_vector in taxi_vectors:
        similarity_score = cosine_similarity(taxi_vector, label_vector)
        if similarity_score > threshold:
            similar_words_set.add(label)
            
similar_words_list = list(similar_words_set)

In [ ]:
len(similar_words_list)

In [ ]:
with open(f'filtered_labels_{len(label)}.txt', 'w') as file:
    # for label in similar_words_list:
    file.write(label + '\n')

In [ ]:
del ft

## Data Loading


In [3]:
data = pd.read_csv('data.csv')
len(data)

83399

In [4]:
gloss_to_label = data[['label', 'clean label']].set_index('label').to_dict()['clean label']
gloss_to_label['None'] = 'None'
len(gloss_to_label)

2732

In [3]:
labels = []
with open('Test/filtered.txt', 'r') as file:
    labels = file.read().splitlines()
len(labels)

530

In [6]:
filtered_hand = list(range(21))

filtered_pose = [11, 12, 13, 14, 15, 16]

filtered_face = [4, 6, 8, 9, 33, 37, 40, 46, 52, 55, 61, 70, 80, 82, 84,
                 87, 88, 91, 105, 107, 133, 145, 154, 157, 159, 161, 163,
                 263, 267, 270, 276, 282, 285, 291, 300, 310, 312, 314, 317,
                 318, 321, 334, 336, 362, 374, 381, 384, 386, 388, 390, 468, 473]

num_hand = len(filtered_hand)
num_pose = len(filtered_pose)
num_face = len(filtered_face)

In [7]:
landmarks = (
    [x for x in filtered_hand] +
    [x + num_hand for x in filtered_hand] +
    [x + num_hand * 2 for x in filtered_pose] +
    [x + num_hand * 2 + num_pose for x in filtered_face]
)

print(landmarks)
print(f'\nTotal Number: {len(landmarks)}')

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 53, 54, 55, 56, 57, 58, 52, 54, 56, 57, 81, 85, 88, 94, 100, 103, 109, 118, 128, 130, 132, 135, 136, 139, 153, 155, 181, 193, 202, 205, 207, 209, 211, 311, 315, 318, 324, 330, 333, 339, 348, 358, 360, 362, 365, 366, 369, 382, 384, 410, 422, 429, 432, 434, 436, 438, 516, 521]

Total Number: 100


In [59]:
def load_data(split, *paths, labels=None, landmarks=None):
    if labels is None:
        labels = set(data['clean label'].unique().tolist())

    filtered_data = data[(data['split'] == split.lower()) & (data['clean label'].isin(labels))]
    label_dict = dict(zip(filtered_data['video'].str.replace('.mp4', ''), filtered_data['label']))
    video_set = set(label_dict.keys())

    X, Y = [], []
    for path in paths:
        landmarks_dict = np.load(path, allow_pickle=True)
        keys = [k for k in landmarks_dict.keys() if k in video_set]

        if landmarks is None:
            landmarks = list(range(landmarks_dict[keys[0]].shape[1]))

        split_name = {'train': 'Training', 'test': 'Test', 'val': 'Validation'}[split.lower()]
        print(f"\nLoading {split_name} data from {path}:")
        
        for k in tqdm(keys, ncols=100):
            num_frames, num_cols, _ = landmarks_dict[k].shape

            landmark_array = np.zeros((num_frames, 100, 3)) # change with the number of landmarks used
            for i, landmark in enumerate(landmarks):
                if landmark < num_cols:
                    landmark_array[:, i, :] = landmarks_dict[k][:, landmark, :]

            X.append(landmark_array)
            Y.append(label_dict.get(k, None))

    return X, Y


In [60]:
X_train, Y_train = load_data('train', 'landmarks.npz',
                             labels=labels, landmarks=landmarks)

X_val, Y_val = load_data('val', 'landmarks.npz',
                             labels=labels, landmarks=landmarks)

X_test, Y_test = load_data('test', 'landmarks.npz',
                             labels=labels, landmarks=landmarks)

C:\Users\ABHISHEK\AppData\Local\Temp\ipykernel_22568\2021755486.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  label_dict = dict(zip(filtered_data['video'].str.replace('.mp4', ''), filtered_data['label']))



Loading Training data from landmarks.npz:


100%|███████████████████████████████████████████████████████████| 8623/8623 [32:07<00:00,  4.47it/s]



Loading Validation data from landmarks.npz:


100%|███████████████████████████████████████████████████████████| 7169/7169 [25:15<00:00,  4.73it/s]



Loading Test data from landmarks.npz:


100%|███████████████████████████████████████████████████████████| 2238/2238 [08:46<00:00,  4.25it/s]


In [61]:
len(np.unique(Y_train)), len(np.unique(Y_val)), len(np.unique(Y_test))

(589, 589, 589)

In [62]:
np.all(np.in1d(np.unique(Y_val), np.unique(Y_train))),\
np.all(np.in1d(np.unique(Y_test), np.unique(Y_train)))

(True, True)

## Data Preprocessing

### Data Augmentation


In [63]:
def rotate(data, rotation_matrix):
    frames, landmarks, _ = data.shape
    center = np.array([0.5, 0.5, 0])
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data = data.reshape(-1, 3)
    data[non_zero] -= center
    data[non_zero] = np.dot(data[non_zero], rotation_matrix.T)
    data[non_zero] += center
    data = data.reshape(frames, landmarks, 3)
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def rotate_z(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])
    return rotate(data, rotation_matrix)

def rotate_y(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    return rotate(data, rotation_matrix)

def rotate_x(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta), np.cos(theta)]
    ])
    return rotate(data, rotation_matrix)

In [64]:
def zoom(data):
    factor = np.random.uniform(0.8, 1.2)
    center = np.array([0.5, 0.5])
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data[non_zero[:, 0], non_zero[:, 1], :2] = (
        (data[non_zero[:, 0], non_zero[:, 1], :2] - center) * factor + center
    )
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def shift(data):
    x_shift = np.random.uniform(-0.2, 0.2)
    y_shift = np.random.uniform(-0.2, 0.2)
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data[non_zero[:, 0], non_zero[:, 1], 0] += x_shift
    data[non_zero[:, 0], non_zero[:, 1], 1] += y_shift
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def mask(data):
    frames, landmarks, _ = data.shape
    num_hands = int(0.3 * 42)
    num_rest = int(0.6 * (landmarks - 42))

    mask = np.zeros(landmarks, dtype=bool)
    indices = np.concatenate([
        np.random.choice(42, num_hands, replace=False),
        np.random.choice(landmarks - 42, num_rest, replace=False) + 42
    ])
    mask[indices] = True
    data[:, mask] = 0
    return data

def speedup(data):
    return data[::2]

In [65]:
def apply_augmentations(data):
    aug_functions = [rotate_x, rotate_y, rotate_z, zoom, shift, mask, speedup]
    np.random.shuffle(aug_functions)
    counter = 0
    for fun in aug_functions:
        if np.random.rand() < 0.5:
            data = fun(data)
            counter += 1
    
    if counter == 0:
        data = apply_augmentations(data)

    return data

In [66]:
def augment(X, Y, num=None):
    X_aug = X.copy()
    Y_aug = Y.copy()
    
    if num == None:
        for i in tqdm(range(len(Y)), ncols=100):
            num_aug = np.random.choice([1, 2, 3])
            for n in range(num_aug):
                X_aug.append(apply_augmentations(X[i].copy()))
                Y_aug.append(Y[i])
    elif num > 0:
        for i in tqdm(range(len(Y)), ncols=100):
            for n in range(num):
                X_aug.append(apply_augmentations(X[i].copy()))
                Y_aug.append(Y[i])

    return X_aug, Y_aug

In [67]:
X_train, Y_train = augment(X_train, Y_train, num=2)

100%|███████████████████████████████████████████████████████████| 8623/8623 [01:39<00:00, 87.03it/s]


In [68]:
len(X_train), len(X_train[0]), len(X_train[0][0]), len(X_train[0][0][0])

(25869, 66, 100, 3)

### Permutation

In [69]:
seed_value = 42
np.random.seed(seed_value)

permutation_train = np.random.permutation(len(Y_train))
permutation_val = np.random.permutation(len(Y_val))
permutation_test = np.random.permutation(len(Y_test))

X_train = [X_train[i] for i in permutation_train]
Y_train = [Y_train[i] for i in permutation_train]

X_val = [X_val[i] for i in permutation_val]
Y_val = [Y_val[i] for i in permutation_val]

X_test = [X_test[i] for i in permutation_test]
Y_test = [Y_test[i] for i in permutation_test]

In [70]:
max(len(x) for x in X_train), max(len(x) for x in X_val), max(len(x) for x in X_test)

(664, 373, 208)

In [71]:
sum(len(x) for x in X_train) / len(X_train),\
sum(len(x) for x in X_val) / len(X_val),\
sum(len(x) for x in X_test) / len(X_test)

(69.53098302988133, 75.43004603152463, 85.20464700625558)

### Padding

In [72]:
def padding(X, Y, length=None, pad=0):
    if length is None:
        length = max(len(x) for x in X)
    
    X_padded = []
    for x in X:
        if len(x) > length:
            X_padded.append(x[:length]) #truncate
        else:
            pad_length = length - len(x)
            X_padded.append(np.pad(
                x, ((0, pad_length), (0, 0), (0, 0)),
                mode='constant', constant_values=pad
            ))
            
    X_padded = np.array(X_padded)
    Y = np.array(Y)
    return X_padded, Y

In [73]:
X_train, Y_train = padding(X_train, Y_train, length=120, pad=-100)
X_val, Y_val = padding(X_val, Y_val, length=120, pad=-100)
X_test, Y_test = padding(X_test, Y_test, length=120, pad=-100)

In [74]:
X_train.shape, X_val.shape, X_test.shape

((25869, 120, 100, 3), (7169, 120, 100, 3), (2238, 120, 100, 3))

### Sequencing

In [75]:
def sequences(X, Y, length=30, step=1, pad=0):
    X_sequences = []
    Y_sequences = []

    for inputs, label in zip(X, Y):
        num = inputs.shape[0]

        if num < length:
            padding = length - num
            inputs = np.pad(
            inputs, ((0, padding), (0, 0), (0, 0)),
            mode='constant', constant_values=pad
            )
            num = length

        for start in range(0, num - length + 1, step):
            end = start + length
            sequence = inputs[start:end]
            X_sequences.append(sequence)
            Y_sequences.append(label)

    X_sequences = np.array(X_sequences)
    Y_sequences = np.array(Y_sequences)
    return X_sequences, Y_sequences

In [76]:
X_train, Y_train = sequences(X_train, Y_train, length=60, step=20, pad=-100)
X_val, Y_val = sequences(X_val, Y_val, length=60, step=20, pad=-100)
X_test, Y_test = sequences(X_test, Y_test, length=60, step=20, pad=-100)

In [77]:
X_train.shape, X_val.shape, X_test.shape

((103476, 60, 100, 3), (28676, 60, 100, 3), (8952, 60, 100, 3))

### Interpolation

In [78]:
def interpolate(X, Y, length=100):
    X_interpolated = [np.apply_along_axis(lambda arr: np.interp(np.linspace(0, 1, length),
                                                                np.linspace(0, 1, arr.shape[0]),
                                                                arr), axis=0, arr=x) for x in X]
    
    X = np.array(X_interpolated)
    Y = np.array(Y)
    return X, Y

In [79]:
X_train, Y_train = interpolate(X_train, Y_train, length=100)
X_val, Y_val = interpolate(X_val, Y_val, length=100)
X_test, Y_test = interpolate(X_test, Y_test, length=100)

MemoryError: Unable to allocate 23.1 GiB for an array with shape (103476, 100, 100, 3) and data type float64

In [ ]:
X_train.shape, X_val.shape, X_test.shape

### Padding from the center

In [82]:
def padding(X, Y, length=None, pad=0):
    if length is None:
        length = max(len(x) for x in X)
    
    X_padded = []
    for x in tqdm(X):
        if len(x) > length:
            start = (len(x) - length) // 2
            end = start + length
            X_padded.append(x[start:end])
        else:
            pad_before = (length - len(x)) // 2
            pad_after = length - len(x) - pad_before
            X_padded.append(np.pad(
                x, ((pad_before, pad_after), (0, 0), (0, 0)),
                mode='constant', constant_values=pad
            ))
            
    X_padded = np.array(X_padded)
    Y = np.array(Y)
    return X_padded, Y

In [83]:
X_train, Y_train = padding(X_train, Y_train, length=120, pad=0)
X_val, Y_val = padding(X_val, Y_val, length=120, pad=0)
X_test, Y_test = padding(X_test, Y_test, length=120, pad=0)

 22%|██▏       | 22878/103476 [00:22<00:56, 1425.64it/s]

In [ ]:
X_train.shape, X_val.shape, X_test.shape

## Label Encoding


In [28]:
gloss_mapping = {gloss: idx for idx, gloss in enumerate(glosses)}
with open(f'Test/gloss_mapping.json', 'w') as json_file:
    json.dump(gloss_mapping, json_file, indent=4)
    
index_gloss_mapping = {idx: gloss for gloss, idx in gloss_mapping.items()}
with open(f'Test/index_gloss_mapping.json', 'w') as json_file:
    json.dump(index_gloss_mapping, json_file, indent=4)
    
index_label_mapping = {idx: gloss_to_label[gloss] for gloss, idx in gloss_mapping.items()}
with open(f'Test/index_label_mapping.json', 'w') as json_file:
    json.dump(index_label_mapping, json_file, indent=4)

In [29]:
Y_train= np.array([gloss_mapping[label] for label in Y_train])
Y_val = np.array([gloss_mapping[label] for label in Y_val])
Y_test = np.array([gloss_mapping[label] for label in Y_test])

In [30]:
Y_train

array([  0,   1,   2, ..., 117, 231, 212])

## Transformer Model 


In [32]:
!pip install -q tensorflow-addons
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
gc.collect()
tf.keras.backend.clear_session()
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
physical_devices

### Model Class

In [34]:
class ECA(tf.keras.layers.Layer):
    def __init__(self, kernel_size=5, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)

    def call(self, inputs, mask=None):
        nn = tf.keras.layers.GlobalAveragePooling1D()(inputs, mask=mask)
        nn = tf.expand_dims(nn, -1)
        nn = self.conv(nn)
        nn = tf.squeeze(nn, -1)
        nn = tf.nn.sigmoid(nn)
        nn = nn[:,None,:]
        return inputs * nn

class LateDropout(tf.keras.layers.Layer):
    def __init__(self, rate, noise_shape=None, start_step=0, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.rate = rate
        self.start_step = start_step
        self.dropout = tf.keras.layers.Dropout(rate, noise_shape=noise_shape)
      
    def build(self, input_shape):
        super().build(input_shape)
        agg = tf.VariableAggregation.ONLY_FIRST_REPLICA
        self._train_counter = tf.Variable(0, dtype="int64", aggregation=agg, trainable=False)

    def call(self, inputs, training=False):
        x = tf.cond(self._train_counter < self.start_step, lambda:inputs, lambda:self.dropout(inputs, training=training))
        if training:
            self._train_counter.assign_add(1)
        return x

class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self, 
        kernel_size=17,
        dilation_rate=1,
        use_bias=False,
        depthwise_initializer='glorot_uniform',
        name='', **kwargs):
        super().__init__(name=name,**kwargs)
        self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(
                            kernel_size,
                            strides=1,
                            dilation_rate=dilation_rate,
                            padding='valid',
                            use_bias=use_bias,
                            depthwise_initializer=depthwise_initializer,
                            name=name + '_dwconv')
        self.supports_masking = True
        
    def call(self, inputs):
        x = self.causal_pad(inputs)
        x = self.dw_conv(x)
        return x

def Conv1DBlock(channel_size,
          kernel_size,
          dilation_rate=1,
          drop_rate=0.0,
          expand_ratio=2,
          se_ratio=0.25,
          activation='swish',
          name=None):

    if name is None:
        name = str(tf.keras.backend.get_uid("mbblock"))
    # Expansion phase
    def apply(inputs):
        channels_in = tf.keras.backend.int_shape(inputs)[-1]
        channels_expand = channels_in * expand_ratio

        skip = inputs

        x = tf.keras.layers.Dense(
            channels_expand,
            use_bias=True,
            activation=activation,
            name=name + '_expand_conv')(inputs)

        # Depthwise Convolution
        x = CausalDWConv1D(kernel_size,
            dilation_rate=dilation_rate,
            use_bias=False,
            name=name + '_dwconv')(x)

        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)

        x  = ECA()(x)

        x = tf.keras.layers.Dense(
            channel_size,
            use_bias=True,
            name=name + '_project_conv')(x)

        if drop_rate > 0:
            x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1), name=name + '_drop')(x)

        if (channels_in == channel_size):
            x = tf.keras.layers.add([x, skip], name=name + '_add')
        return x

    return apply

In [35]:
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, dim=256, num_heads=4, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.scale = self.dim ** -0.5
        self.num_heads = num_heads
        self.qkv = tf.keras.layers.Dense(3 * dim, use_bias=False)
        self.drop1 = tf.keras.layers.Dropout(dropout)
        self.proj = tf.keras.layers.Dense(dim, use_bias=False)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        qkv = self.qkv(inputs)
        qkv = tf.keras.layers.Permute((2, 1, 3))(tf.keras.layers.Reshape((-1, self.num_heads, self.dim * 3 // self.num_heads))(qkv))
        q, k, v = tf.split(qkv, [self.dim // self.num_heads] * 3, axis=-1)

        attn = tf.matmul(q, k, transpose_b=True) * self.scale

        if mask is not None:
            mask = mask[:, None, None, :]

        attn = tf.keras.layers.Softmax(axis=-1)(attn, mask=mask)
        attn = self.drop1(attn)

        x = attn @ v
        x = tf.keras.layers.Reshape((-1, self.dim))(tf.keras.layers.Permute((2, 1, 3))(x))
        x = self.proj(x)
        return x


def TransformerBlock(dim=256, num_heads=4, expand=4, attn_dropout=0.2, drop_rate=0.2, activation='swish'):
    def apply(inputs):
        x = inputs
        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = MultiHeadSelfAttention(dim=dim,num_heads=num_heads,dropout=attn_dropout)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([inputs, x])
        attn_out = x

        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = tf.keras.layers.Dense(dim*expand, use_bias=False, activation=activation)(x)
        x = tf.keras.layers.Dense(dim, use_bias=False)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([attn_out, x])
        return x
    return apply

In [36]:
class OneCycleLR(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self,
                lr=1e-4,
                epochs=10,
                steps_per_epoch=100,
                steps_per_update=1,
                resume_epoch=0,
                decay_epochs=10,
                sustain_epochs=0,
                warmup_epochs=0,
                lr_start=0,
                lr_min=0,
                warmup_type='linear',
                decay_type='cosine',
                **kwargs):
        
        super().__init__(**kwargs)
        self.lr = float(lr)
        self.epochs = float(epochs)
        self.steps_per_update = float(steps_per_update)
        self.resume_epoch = float(resume_epoch)
        self.steps_per_epoch = float(steps_per_epoch)
        self.decay_epochs = float(decay_epochs)
        self.sustain_epochs = float(sustain_epochs)
        self.warmup_epochs = float(warmup_epochs)
        self.lr_start = float(lr_start)
        self.lr_min = float(lr_min)
        self.decay_type = decay_type
        self.warmup_type = warmup_type
        

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        total_steps = self.epochs * self.steps_per_epoch
        warmup_steps = self.warmup_epochs * self.steps_per_epoch
        sustain_steps = self.sustain_epochs * self.steps_per_epoch
        decay_steps = self.decay_epochs * self.steps_per_epoch

        if self.resume_epoch > 0:
            step = step + self.resume_epoch * self.steps_per_epoch

        step = tf.cond(step > decay_steps, lambda :decay_steps, lambda :step)
        step = tf.math.truediv(step, self.steps_per_update) * self.steps_per_update

        warmup_cond = step < warmup_steps
        decay_cond = step >= (warmup_steps + sustain_steps)
        
        if self.warmup_type == 'linear':
            lr = tf.cond(warmup_cond, lambda: tf.math.divide_no_nan(self.lr-self.lr_start , warmup_steps) * step + self.lr_start, lambda: self.lr)
        elif self.warmup_type == 'exponential':
            factor = tf.pow(self.lr_start, 1/warmup_steps)
            lr = tf.cond(warmup_cond, lambda: (self.lr - self.lr_start) * factor**(warmup_steps - step) + self.lr_start, lambda: self.lr)
        elif self.warmup_type == 'cosine':
            lr = tf.cond(warmup_cond, lambda: 0.5 * (self.lr - self.lr_start) * (1 + tf.cos(3.14159265359 * (warmup_steps - step)  / warmup_steps)) + self.lr_start, lambda:self.lr)
        else:
            raise NotImplementedError
                    
        
        if self.decay_type == 'linear':
            lr = tf.cond(decay_cond, lambda: self.lr + (self.lr_min-self.lr)/(decay_steps - warmup_steps - sustain_steps)*(step - warmup_steps - sustain_steps), lambda:lr)
        elif self.decay_type == 'exponential':
            factor = tf.pow(self.lr_min, 1/(decay_steps - warmup_steps - sustain_steps))
            lr = tf.cond(decay_cond, lambda: (self.lr - self.lr_min) * factor**(step - warmup_steps - sustain_steps) + self.lr_min, lambda:lr)
        elif self.decay_type == 'cosine':
            lr = tf.cond(decay_cond, lambda: 0.5 * (self.lr - self.lr_min) * (1 + tf.cos(3.14159265359 * (step - warmup_steps - sustain_steps) / (decay_steps - warmup_steps - sustain_steps))) + self.lr_min, lambda:lr)
        else:
            raise NotImplementedError
            
        return lr

    def plot(self):
        step = max(1, int(self.epochs*self.steps_per_epoch)//1000) #1 for total_steps < 1000, total_steps//1000 else
        eps = list(range(0,int(self.epochs*self.steps_per_epoch),step))
        learning_rates = [self(x) for x in eps]
        plt.scatter(eps,learning_rates,2)
        plt.show()
        
    def get_config(self):
        config = {
            'lr': self.lr,
            'epochs': self.epochs,
            'steps_per_update': self.steps_per_update,
            'resume_epoch': self.resume_epoch,
            'steps_per_epoch': self.steps_per_epoch,
            'decay_epochs': self.decay_epochs,
            'sustain_epochs': self.sustain_epochs,
            'warmup_epochs': self.warmup_epochs,
            'lr_start': self.lr_start,
            'lr_min': self.lr_min,
            'decay_type': self.decay_type,
            'warmup_type': self.warmup_type
        }
        return config

### Model Functiom 
- Input, output shape can be easily changed depending on the number of landmarks used, length etc.

In [37]:
def get_model(max_len=120, dropout_step=0, dim=192):
    inp = tf.keras.layers.Input(X_train[0].shape, name='input')
    x = inp
    x = tf.reshape(x, [-1, X_train[0].shape[0], X_train[0].shape[1] * X_train[0].shape[2]])

    ksize = 17
    x = tf.keras.layers.Dense(dim, use_bias=False,name='stem_conv')(x)
    x = tf.keras.layers.BatchNormalization(momentum=0.95,name='stem_bn')(x)

    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = TransformerBlock(dim,expand=2)(x)

    x = Conv1DBlock(dim,ksize,drop_rate=0.25)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.25)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.25)(x)
    x = TransformerBlock(dim,expand=2)(x)

    x = tf.keras.layers.Dense(512,activation=None,name='top_conv')(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = LateDropout(0.8, start_step=dropout_step)(x)
    x = tf.keras.layers.Dense(len(glosses),name='classifier')(x)
    
    model = tf.keras.models.Model(inputs=inp, outputs=x)
    
    schedule = OneCycleLR(4e-3, 200, steps_per_epoch=192, decay_epochs=200, lr_min=1e-6, decay_type='cosine', warmup_type='linear')
    decay_schedule = OneCycleLR(4e-4, 200, steps_per_epoch=192, decay_epochs=200, lr_min=1e-7, decay_type='cosine', warmup_type='linear')
    optimizer = tfa.optimizers.RectifiedAdam(learning_rate=schedule, weight_decay=decay_schedule, sma_threshold=4)
    optimizer = tfa.optimizers.Lookahead(optimizer,sync_period=5)
    model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    metrics=['accuracy']
    )
    return model

In [38]:
tf.keras.backend.clear_session()
model = get_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 120, 100, 3)]        0         []                            
                                                                                                  
 tf.reshape (TFOpLambda)     (None, 120, 300)             0         ['input[0][0]']               
                                                                                                  
 stem_conv (Dense)           (None, 120, 192)             57600     ['tf.reshape[0][0]']          
                                                                                                  
 stem_bn (BatchNormalizatio  (None, 120, 192)             768       ['stem_conv[0][0]']           
 n)                                                                                           

### Fitting the Model

In [39]:
acc_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_accuracy_weights.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
)

loss_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_loss_weights.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=50,
    restore_best_weights=True
)
    

hist = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=500,
    batch_size=256,
    callbacks=[acc_checkpoint, loss_checkpoint, early_stopping]
)
    

Epoch 1/500
47/47 [==============================] - 71s 430ms/step - loss: 6.6877 - accuracy: 0.0034 - val_loss: 5.6211 - val_accuracy: 0.0049
Epoch 2/500
47/47 [==============================] - 18s 373ms/step - loss: 5.9190 - accuracy: 0.0047 - val_loss: 5.5801 - val_accuracy: 0.0052
Epoch 3/500
47/47 [==============================] - 17s 373ms/step - loss: 5.7498 - accuracy: 0.0053 - val_loss: 5.5045 - val_accuracy: 0.0122
Epoch 4/500
47/47 [==============================] - 17s 371ms/step - loss: 5.6408 - accuracy: 0.0089 - val_loss: 5.4094 - val_accuracy: 0.0119
Epoch 5/500
47/47 [==============================] - 18s 376ms/step - loss: 5.5167 - accuracy: 0.0114 - val_loss: 5.2174 - val_accuracy: 0.0296
Epoch 6/500
47/47 [==============================] - 18s 376ms/step - loss: 5.3905 - accuracy: 0.0155 - val_loss: 5.0957 - val_accuracy: 0.0256
Epoch 7/500
47/47 [==============================] - 18s 379ms/step - loss: 5.2670 - accuracy: 0.0182 - val_loss: 4.9520 - val_accuracy:

In [41]:
model.evaluate(X_val, Y_val)

103/103 [==============================] - 2s 21ms/step - loss: 2.4286 - accuracy: 0.6721


[2.4286136627197266, 0.6720561385154724]

In [42]:
model.evaluate(X_test, Y_test)

32/32 [==============================] - 1s 20ms/step - loss: 1.5563 - accuracy: 0.7669


[1.5562608242034912, 0.766865074634552]

### Optimized Model
- Saving an optimized version of the model for web and app applications
- We will be using tflite model for our deployment

In [44]:
name = 'model'
model_filepath = 'Models'
os.makedirs(model_filepath, exist_ok=True)

model.save(model_filepath + f'/{name}.h5')
model.save(model_filepath + f'/{name}')
converter = tf.lite.TFLiteConverter.from_saved_model(model_filepath + f'/{name}')
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS,
  tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()
with open(model_filepath + f'/{name}.tflite', 'wb') as f:
    f.write(tflite_model)


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 89, Total Ops 467, % non-converted = 19.06 %
 * 89 ARITH ops

- arith.constant:   89 occurrences  (f32: 67, i32: 22)



  (f32: 28)
  (f32: 4)
  (i32: 22)
  (f32: 6)
  (f32: 6)
  (f32: 18)
  (f32: 23)
  (i32: 44)
  (f32: 14)
  (f32: 7)
  (f32: 21)

  (i32: 26)
  (f32: 6)
  (i32: 44)
  (f32: 61)
  (i32: 26)
  (f32: 2)
  (f32: 2)
  (f32: 6, i32: 4)
  (f32: 4)


---

### TFLite model Evaluation

In [58]:
model_path = 'Models\model(old-face_landmark).tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']
print(f'Input Shape: {input_shape}, Output Shape: {output_shape}')

correct_gloss = 0
correct_label = 0
for i in tqdm(range(len(Y_test)), ncols=100):
    input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    prediction = output.argmax()
    if index_label_mapping[prediction] == index_label_mapping[Y_test[i]]:
        correct_label += 1
        if prediction == Y_test[i]:
            correct_gloss += 1
gloss_accuracy = correct_gloss / len(Y_test)
label_accuracy = correct_label / len(Y_test)
print(f'Gloss Accuracy: {gloss_accuracy:.4f}')
print(f'Label Accuracy: {label_accuracy:.4f}')

Input Shape: [  1 120 100   3], Output Shape: [  1 269]


100%|███████████████████████████████████████████████████████████| 1008/1008 [00:16<00:00, 61.83it/s]

Gloss Accuracy: 0.7669
Label Accuracy: 0.7927


## Confidence range and Certainity

In [ ]:
Y_pred = []

for i in tqdm(range(len(Y_val))):
    input_data = np.expand_dims(X_val[i], axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    Y_pred.append(output_data[0])

Y_pred = np.array(Y_pred)
Y_pred.shape

In [ ]:
class_idxs = np.argmax(Y_pred, axis=1)
class_idxs.shape, np.unique(class_idxs).shape

In [ ]:
# Calculate average prediction for each class
avg_positive = np.array([np.mean(Y_pred[class_idxs == cls] + 1e-6, axis=0)[cls] for cls in range(len(glosses))])
avg_negative = np.array([np.mean(Y_pred[class_idxs != cls] + 1e-6, axis=0)[cls] for cls in range(len(glosses))])

# Calculate standard deviation for each class
std_positive = np.array([np.std(Y_pred[class_idxs == cls] + 1e-6, axis=0)[cls] for cls in range(len(glosses))])
std_negative = np.array([np.std(Y_pred[class_idxs != cls] + 1e-6, axis=0)[cls] for cls in range(len(glosses))])

# Calculate estimated range for last layer neurons
range_positive_lower = avg_positive - 2 * std_positive
range_positive_upper = avg_positive + 2 * std_positive
range_positive = np.stack((range_positive_lower, avg_positive, range_positive_upper), axis=1)

range_negative_lower = avg_negative - 2 * std_negative
range_negative_upper = avg_negative + 2 * std_negative
range_negative = np.stack((range_negative_lower, avg_negative, range_negative_upper), axis=1)

In [ ]:
f'Shape: {range_positive.shape}, Examples: {range_positive[5]}, {range_positive[9]}, {range_positive[15]}'

In [ ]:
np.save('Test/range_positive.npy', range_positive)
np.save('Test/range_negative.npy', range_negative)

In [ ]:
def calculate_certainty(pred, pos_range, neg_range):
    z_score_true = abs(pred - pos_range[1]) / (pos_range[2] - pos_range[0])
    z_score_false = abs(pred - neg_range[1]) / (neg_range[2] - neg_range[0])
    return z_score_false / (z_score_true + z_score_false)

In [ ]:
print(f'{calculate_certainty(50, range_positive[80], range_negative[80]) * 100:.2f} %')